## Use this code with env pytorch_deploy

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pyrealsense2 as rs
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from cv2 import VideoWriter, VideoWriter_fourcc
import pandas as pd
import torch
from scipy import signal, ndimage, spatial
from functions import *

In [3]:
from face_alignment import api as face_alignment
from face_alignment.models import FAN, ResNetDepth
from face_alignment.detection.sfd import sfd_detector

def load_weights(model, filename):
    sd = torch.load(filename, map_location=lambda storage, loc: storage)
    names = set(model.state_dict().keys())
    for n in list(sd.keys()): 
        if n not in names and n+'_raw' in names:
            if n+'_raw' not in sd: sd[n+'_raw'] = sd[n]
            del sd[n]
    model.load_state_dict(sd)
    
    
def load_weights_depth(model, filename):
    sd = torch.load(filename, map_location=lambda storage, loc: storage)
    depth_dict = {
        k.replace('module.', ''): v for k,
        v in sd['state_dict'].items()}
    
    model.load_state_dict(depth_dict)
    

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#face_alignment_model = r"./models/2DFAN4-11f355bf06.pth.tar" 
#face_alignment_model = r"./models/model_finetuned_mix.pth"
#Face alignement
#network_size = 4
#face_alignment_net = FAN(network_size)
#load_weights(face_alignment_net,face_alignment_model)
#face_alignment_net.to(device)
#face_alignment_net.eval()

#face_alignment_model = r"./models/netlast_epoch.pth" 
#face_alignment_model = r"./models/model_finetuned_mix.pth"


#Face alignement
#network_size = 4
#face_alignment_net = FAN(network_size)
#load_weights(face_alignment_net,face_alignment_model)
#face_alignment_net.to(device)
#face_alignment_net.eval()


face_detector_model = r"/home/deniz/Code/ClinicalScore/models/s3fd-619a316812.pth"
face_detection_net = sfd_detector.SFDDetector(device=device, path_to_detector=face_detector_model, verbose=False)




#Face alignement
face_alignment_model_finetuned = r"/home/deniz/Code/ClinicalScore/models/model_finetuned_mix_wingloss_w15.pth"
network_size = 4
face_alignment_net = FAN(network_size)
load_weights(face_alignment_net,face_alignment_model_finetuned)
face_alignment_net.to(device)
face_alignment_net.eval()


# depth_network = ResNetDepth();
# depth_file = r"./models/depth-2a464da4ea.pth.tar"
# load_weights_depth(depth_network, depth_file);
# depth_network.to(device);
# depth_network.eval();
# print('done')

/home/deniz/.pyenv/versions/reqtxt/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


FAN(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): ConvBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (downsample): Sequential(
      (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (conv3): ConvBlock(
    (

In [4]:
print(device)

cpu


In [5]:
print(torch.cuda.is_available())

False


In [8]:

population = ['ALS', 'PLS', 'Kennedys', 'Post-stroke', 'Healthy_Control']

tasks = ['BBP_NORMAL', 'NSM_OPEN', 'NSM_SPREAD', 'NSM_BIGSMILE' , 'RST_REST', 'NSM_KISS', 'NSM_PUCKER', 'DDK_PA']


df_cols = ['BAG_Files', 'COLOR_Files', 'cvs_frame_info_Files', 'output_Files']
df = pd.DataFrame(columns=df_cols)



base_path = '/media/deniz/easystore/facial_tracking_data_2021'

sub_folders = '/02/RSSDK'

output_sub_folders = '/02/LANDMARKS/Deniz_FineTunedFAN'

In [10]:
bag_path = r'/media/deniz/easystore/facial_tracking_data_2021/all_bag_files_for_clinical_score.csv'
bag_df = pd.read_csv(bag_path)
bag_df = bag_df.drop("Unnamed: 0", axis=1)
bag_df.head()

,subID,Date,task,bag_path
0,AF12,20180731,PUCKER,/media/deniz/easystore/facial_tracking_data_20...
1,AF12,20180731,OPEN,/media/deniz/easystore/facial_tracking_data_20...
2,AF12,20180731,SPREAD,/media/deniz/easystore/facial_tracking_data_20...
3,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...
4,AF12,20180731,REST,/media/deniz/easystore/facial_tracking_data_20...


In [32]:
bag_df['bag_path'][0]

'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF12_SBK_20180731/02/RSSDK/AF12_02_20180731_NSM_KISS.bag'

In [ ]:
# for each bag file that has not been processed by the finetuned fan model and has no 3D.csv generated, process now
sufix = '_landmarksFiltered3D.csv'
output_subfolder = 'LANDMARKS/Diego_FaceData/landmarks_finetuned_w15'

for index, row in bag_df.iterrows():
    
    BAG_File = row['bag_path']
    splited_name = row['bag_path'].split('/')
    name = splited_name[-1][:-4]
    root_path = '/'.join(splited_name[:-2])
    
    folder  = os.path.join(root_path,output_subfolder)
    csv_name = os.path.join(folder, name + sufix)
    
    if os.path.exists(csv_name):
        continue 
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        try: 
            
            color_file, cvs_frame_info = get_color_video(BAG_File)

            landmark_file = find_landmarks(BAG_File, device, color_file, cvs_frame_info,
                                           localize_face=60,
                                           sufix = folder,
                                           fix_head_position=10,
                                           face_alignment_net=face_alignment_net, 
                                           face_detection_net=face_detection_net)

            
            landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=False)

            
            get3dlandmarks(BAG_File, cvs_frame_info, landmark_file_filt)
            
        except: 
            continue

    
    
    
    
    

# Random Testing

In [ ]:

root_path = '/'.join(bag_df['bag_path'][0].split('/')[:-2])

folder  = os.path.join(root_path,output_subfolder)
print(folder)
os.path.exists(folder)

In [ ]:


name = bag_df['bag_path'][0].split('/')[-1][:-4]
name = name + sufix
print(name)
os.path.join(folder,name)

In [ ]:




for p in population:
    data_dir = os.path.join(base_path, p) 
    print('working on pop: ', p)
    
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            print(dirs)
            raw_dir = os.path.join(data_dir, dirs) + sub_folders_1
           
            if not os.path.exists(raw_dir):
                rawdata_list = os.listdir(raw_dir)

for root, dirs, files in os.walk(r'E:\FaceData_\video_data', topdown=False):
    
    for name in files:
        if name.endswith('.avi'): # we found a video

                
            if 'new' in root:
                color_file = os.path.join(root,name)
                BAG_File = color_file[:-10]+'.bag'
                cvs_frame_info = color_file[:-10]+'_frameInfoColor.csv' 
                if os.path.exists(BAG_File):
                    
                    #create folder if it doesn't exist 
                    landmarks_folder = os.path.join(root,sufix)
                    if not os.path.exists(landmarks_folder):
                        os.makedirs(landmarks_folder)
                    try: 
                        landmark_file = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarks.csv')
                        if not os.path.exists(landmark_file):

                            landmark_file = find_landmarks(BAG_File, device, color_file, cvs_frame_info,
                                                           localize_face=60,
                                                           sufix = sufix,
                                                           fix_head_position=10,
                                                           face_alignment_net=face_alignment_net, 
                                                           face_detection_net=face_detection_net)
                        
                        landmark_file_filt = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered.csv')
                        if not os.path.exists(landmark_file_filt):
                            landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)
                        
                        landmark_file_filt3D = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered3D.csv')
                        if not os.path.exists(landmark_file_filt3D):
                            get3dlandmarks(BAG_File, cvs_frame_info, landmark_file_filt)
                    except: 
                        continue

                
            if 'old' in root:
                color_file = os.path.join(root,name)
                depth_file = color_file[:-10]+'_depth.avi'
                if os.path.exists(depth_file):
                    #create folder if it doesn't exist 
                    landmarks_folder = os.path.join(root,sufix)
                    if not os.path.exists(landmarks_folder):
                        os.makedirs(landmarks_folder)
                        
                    try: 
                        landmark_file = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarks.csv')
                        if not os.path.exists(landmark_file):
                            landmark_file = find_landmarks_video(device,
                                                       color_file,
                                                       localize_face=60,
                                                       sufix = sufix,
                                                       fix_head_position=10,
                                                       face_alignment_net=face_alignment_net,
                                                       face_detection_net=face_detection_net)

                        landmark_file_filt = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered.csv')
                        if not os.path.exists(landmark_file_filt):
                            landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)

                        landmark_file_filt3D = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered3D.csv')
                        if not os.path.exists(landmark_file_filt3D):
                            get3dlandmarks_video(depth_file, landmark_file_filt)

                    except: 
                        continue

In [9]:
#####
# this section is only for processing bag files

base_path = '/media/deniz/easystore/facial_tracking_data_2021'

sub_folders = '/02/RSSDK'
b_extension = '.bag'

output_sub_folders = '/02/LANDMARKS/Deniz_FineTunedFAN'



for p in population:
    data_dir = os.path.join(base_path, p) 
    print(p)
    
    
    for dirs in os.listdir(data_dir):
        try: 
            data_dir = os.path.join(data_dir, dirs) + sub_folders
            
            list_pop_tasks = os.listdir(data_dir)
            for i in list_pop_tasks:
                for t in tasks: 
                    if (i.endswith(b_extension)) and (t in i):
                        base_name  = i[:-4]
                        color_name = base_name +'_color.avi'
                        csv_name =  base_name + '_frameInfoColor.csv'
                        output = os.path.join(data_dir, dirs) + output_sub_folders
                        
                        new_row = {'BAG_Files': i,'COLOR_Files':color_name, 'cvs_frame_info_Files':csv_name,
                                  'output_Files': output}
                        df = df.append(new_row, ignore_index = True)
        except:
            pass
                        
                    
            
            
    
   


#df['BAG_Files'] = list_tasks_bag

#df['COLOR_Files'] = list_tasks_avi

#df['cvs_frame_info_Files'] = list_tasks_csv
df.head()

ALS
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK/AF03_SBK_20170526/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK/AF03_SBK_20170526/02/RSSDK/AF04_SBK_20170529/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK/AF03_SBK_20170526/02/RSSDK/AF04_SBK_20170529/02/RSSDK/AF05_SBK_20170530/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK/AF03_SBK_20170526/02/RSSDK/AF04_SBK_20170529/02/RSSDK/AF05_SBK_20170530/02/RSSDK/AF06_SBK_20170530/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/RSSDK/AF02_SBK_20170517/02/RSSDK/AF03

/media/deniz/easystore/facial_tracking_data_2021/PLS/April_2021_Unprocessed_bag_files_PLS.csv/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/PLS/April_2021_Unprocessed_bag_files_PLS.csv/02/RSSDK/P002_SBK_20190108/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/PLS/April_2021_Unprocessed_bag_files_PLS.csv/02/RSSDK/P002_SBK_20190108/02/RSSDK/P003_SBK_20190123/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/PLS/April_2021_Unprocessed_bag_files_PLS.csv/02/RSSDK/P002_SBK_20190108/02/RSSDK/P003_SBK_20190123/02/RSSDK/P004_SBK_20190226/02/RSSDK
Kennedys
/media/deniz/easystore/facial_tracking_data_2021/Kennedys/K001_SBK_20180420/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Kennedys/K001_SBK_20180420/02/RSSDK/K001_SBK_20190621/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Kennedys/K001_SBK_20180420/02/RSSDK/K001_SBK_20190621/02/RSSDK/K003_SBK_20190809/02/RSSDK
Post-stroke
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK

/media/deniz/easystore/facial_tracking_data_2021/Post-stroke/OP07_SBK_20191018/02/RSSDK/OP15_SBK_20200218/02/RSSDK/April_2021_Unprocessed_bag_files_PS.csv/02/RSSDK/April_2021_Unprocessed_rssdk_files_PS.csv/02/RSSDK/OP01_SBK_20190301/02/RSSDK/OP01_SBK_20190308/02/RSSDK/OP01_SBK_20190510/02/RSSDK/OP01_SBK_20190613/02/RSSDK/OP02_SBK_20190314/02/RSSDK/OP02_SBK_20190412/02/RSSDK/OP02_SBK_20190517/02/RSSDK/OP03_SBK_20190315/02/RSSDK/OP03_SBK_20190426/02/RSSDK/OP03_SBK_20190524/02/RSSDK/OP04_SBK_20190523/02/RSSDK/OP05_SBK_20190719/02/RSSDK/OP05_SBK_20190830/02/RSSDK/OP05_SBK_20190926/02/RSSDK/OP06_SBK_20190823/02/RSSDK/OP06_SBK_20191004/02/RSSDK/OP06_SBK_20191101/02/RSSDK/OP07_SBK_20190906/02/RSSDK/OP07_SBK_20191121/02/RSSDK/OP08_SBK_20190906/02/RSSDK/OP08_SBK_20191023/02/RSSDK/OP08_SBK_20191119/02/RSSDK/OP09_SBK_20190912/02/RSSDK/OP09_SBK_20191023/02/RSSDK/OP09_SBK_20191206/02/RSSDK/OP10_SBK_20190913/02/RSSDK/OP10_SBK_20191025/02/RSSDK/OP10_SBK_20191121/02/RSSDK/OP11_SBK_20190919/02/RSSDK/OP

/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF15_TRI_20200130/02/RSSDK/April_2021_Unprocessed_bag_files_HC.csv/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF15_TRI_20200130/02/RSSDK/April_2021_Unprocessed_bag_files_HC.csv/02/RSSDK/April_2021_Unprocessed_rssdk_files_HC.csv/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF15_TRI_20200130/02/RSSDK/April_2021_Unprocessed_bag_files_HC.csv/02/RSSDK/April_2021_Unprocessed_rssdk_files_HC.csv/02/RSSDK/NF00_TRI_20170503/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF15_TRI_20200130/02/RSSDK/April_2021_Unprocessed_bag_files_HC.csv/02/RSSDK/April_2021_Unprocessed_rssdk_files_HC.csv/02/RSSDK/NF00_TRI_20170503/02/RSSDK/NF01_SBK_20170515/02/RSSDK
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control/NF15_TRI_20200130/02/RSSDK/April_2021_Unprocessed_bag_files_HC.csv/02/RSSDK/April_2021_Unprocessed_rssdk_files_HC.csv/02/RSSDK/NF00_TRI_2

,BAG_Files,COLOR_Files,cvs_frame_info_Files,output_Files
0,K000_02_20180420_PLT_REST.bag,K000_02_20180420_PLT_REST_color.avi,K000_02_20180420_PLT_REST_frameInfoColor.csv,/media/deniz/easystore/facial_tracking_data_20...
1,K001_02_20180420_BBP_NORMAL.bag,K001_02_20180420_BBP_NORMAL_color.avi,K001_02_20180420_BBP_NORMAL_frameInfoColor.csv,/media/deniz/easystore/facial_tracking_data_20...
2,K001_02_20180420_DDK_PA.bag,K001_02_20180420_DDK_PA_color.avi,K001_02_20180420_DDK_PA_frameInfoColor.csv,/media/deniz/easystore/facial_tracking_data_20...
3,K001_02_20180420_DDK_PATAKA.bag,K001_02_20180420_DDK_PATAKA_color.avi,K001_02_20180420_DDK_PATAKA_frameInfoColor.csv,/media/deniz/easystore/facial_tracking_data_20...
4,K001_02_20180420_TNG_REST.bag,K001_02_20180420_TNG_REST_color.avi,K001_02_20180420_TNG_REST_frameInfoColor.csv,/media/deniz/easystore/facial_tracking_data_20...


In [ ]:
##### ORIGINAL VERSION

for root, dirs, files in os.walk(r'E:\FaceData_\video_data', topdown=False):
    
    for name in files:
        if name.endswith('.avi'): # we found a video

                
            if 'new' in root:
                color_file = os.path.join(root,name)
                BAG_File = color_file[:-10]+'.bag'
                cvs_frame_info = color_file[:-10]+'_frameInfoColor.csv' 
                if os.path.exists(BAG_File):
                    
                    #create folder if it doesn't exist 
                    landmarks_folder = os.path.join(root,sufix)
                    if not os.path.exists(landmarks_folder):
                        os.makedirs(landmarks_folder)
                    try: 
                        landmark_file = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarks.csv')
                        if not os.path.exists(landmark_file):

                            landmark_file = find_landmarks(BAG_File, device, color_file, cvs_frame_info,
                                                           localize_face=60,
                                                           sufix = sufix,
                                                           fix_head_position=10,
                                                           face_alignment_net=face_alignment_net, 
                                                           face_detection_net=face_detection_net)
                        
                        landmark_file_filt = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered.csv')
                        if not os.path.exists(landmark_file_filt):
                            landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)
                        
                        landmark_file_filt3D = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered3D.csv')
                        if not os.path.exists(landmark_file_filt3D):
                            get3dlandmarks(BAG_File, cvs_frame_info, landmark_file_filt)
                    except: 
                        continue

                
            if 'old' in root:
                color_file = os.path.join(root,name)
                depth_file = color_file[:-10]+'_depth.avi'
                if os.path.exists(depth_file):
                    #create folder if it doesn't exist 
                    landmarks_folder = os.path.join(root,sufix)
                    if not os.path.exists(landmarks_folder):
                        os.makedirs(landmarks_folder)
                        
                    try: 
                        landmark_file = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarks.csv')
                        if not os.path.exists(landmark_file):
                            landmark_file = find_landmarks_video(device,
                                                       color_file,
                                                       localize_face=60,
                                                       sufix = sufix,
                                                       fix_head_position=10,
                                                       face_alignment_net=face_alignment_net,
                                                       face_detection_net=face_detection_net)

                        landmark_file_filt = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered.csv')
                        if not os.path.exists(landmark_file_filt):
                            landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)

                        landmark_file_filt3D = os.path.join(os.path.join(root,sufix),name[:-10]+'_landmarksFiltered3D.csv')
                        if not os.path.exists(landmark_file_filt3D):
                            get3dlandmarks_video(depth_file, landmark_file_filt)

                    except: 
                        continue

In [12]:
len(df)

36

In [ ]:
### PROCESSING THE FINE TUNED FAN ON BAG FILES OF INTEREST 

### SAVING DATA IN THE WRITE FOLDER



with_errors = []
processed_directory = os.path.join(path_v, '/processed')
error_directory = os.path.join(path_v, '/with_error')
output_dir = r'/home/deniz/Code/DataQuality/test_video/landmark_finetuned'

for i in range(0, len(df)):
    try: 
        print(df.BAG_Files[i])
        BAG_File = path_v + '/' + Files_v[i]
        #color_file, cvs_frame_info = get_color_video(BAG_File)
        color_file = df.COLOR_Files[i]
        cvs_frame_info = df.cvs_frame_info_Files[i]
        output_dir = df.output_File[i]
        landmark_file = find_landmarks(BAG_File, device, color_file, cvs_frame_info,
                                                               localize_face=60,
                                                               output_dir = output_dir,
                                                               fix_head_position=10,
                                                               face_alignment_net=face_alignment_net, 
                                                               face_detection_net=face_detection_net)
        landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)
        get3dlandmarks(BAG_File, cvs_frame_info, landmark_file_filt)
        processed.append(Files_v[i])
        
    #shutil.move(Files_v[i],processed_directory)
        
    except:
        print(Files_v[i], ' Failed with error!')
        #shutil.move(Files_v[i],error_directory)
        with_errors.append(Files_v[i])

In [13]:
### PROCESSING THE FINE TUNED FAN ON BAG FILES OF INTEREST 

### SAVING DATA IN THE WRITE FOLDER

### ORIGINAL 

with_errors = []
processed_directory = os.path.join(path_v, '/processed')
error_directory = os.path.join(path_v, '/with_error')
output_dir = r'/home/deniz/Code/DataQuality/test_video/landmark_finetuned'

for i in range(0, len(Files_v)):
    try: 
        print(Files_v[i])
        BAG_File = path_v + '/' + Files_v[i]
        color_file, cvs_frame_info = get_color_video(BAG_File)
        landmark_file = find_landmarks(BAG_File, device, color_file, cvs_frame_info,
                                                               localize_face=60,
                                                               output_dir = output_dir,
                                                               fix_head_position=10,
                                                               face_alignment_net=face_alignment_net, 
                                                               face_detection_net=face_detection_net)
        landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)
        get3dlandmarks(BAG_File, cvs_frame_info, landmark_file_filt)
        processed.append(Files_v[i])
        
    #shutil.move(Files_v[i],processed_directory)
        
    except:
        print(Files_v[i], ' Failed with error!')
        #shutil.move(Files_v[i],error_directory)
        with_errors.append(Files_v[i])

AF02_02_20170517_BBP_NORMAL.rssdk
AF02_02_20170517_BBP_NORMAL.rssdk  Failed with error!
NF01_02_20170515_BBP_NORMAL.rssdk
NF01_02_20170515_BBP_NORMAL.rssdk  Failed with error!
AF12_02_20180731_NSM_OPEN.bag
AF12_02_20180731_NSM_OPEN.bag  Failed with error!
AF02_02_20170517_NSM_OPEN.rssdk
AF02_02_20170517_NSM_OPEN.rssdk  Failed with error!
NF08_02_20170615_NSM_OPEN.rssdk
NF08_02_20170615_NSM_OPEN.rssdk  Failed with error!


In [22]:
Files_v.remove('AF12_02_20180731_NSM_OPEN.bag')
Files_v

['AF02_02_20170517_BBP_NORMAL.rssdk',
 'NF01_02_20170515_BBP_NORMAL.rssdk',
 'AF07_02_20170530_BBP_NORMAL.rssdk',
 'AF02_02_20170517_NSM_OPEN.rssdk',
 'NF08_02_20170615_NSM_OPEN.rssdk']

In [29]:
output_dir

'/home/deniz/Code/DataQuality/test_video/landmark_finetuned'

In [32]:
for root, dirs, files in os.walk(r'/home/deniz/Code/DataQuality/test_video/', topdown=False):
    for name in files:
        if name.endswith('color.avi'):
            print(name)
            color_file = os.path.join(root,name)
            depth_file = color_file[:-10]+'_depth.avi'
            if os.path.exists(depth_file):
                #create folder if it doesn't exist 
                landmarks_folder = output_dir
                try: 
                    landmark_file = os.path.join(output_dir,name[:-10]+'_landmarks.csv')
                    if not os.path.exists(landmark_file):
                        landmark_file = find_landmarks_video(device,
                                                   color_file,
                                                   localize_face=60,
                                                   sufix = output_dir,
                                                   fix_head_position=10,
                                                   face_alignment_net=face_alignment_net,
                                                   face_detection_net=face_detection_net)

                    landmark_file_filt = os.path.join(output_dir,name[:-10]+'_landmarksFiltered.csv')
                    if not os.path.exists(landmark_file_filt):
                        landmark_file_filt = smooth_landmarks(landmark_file, color_file,  create_video=True)

                    landmark_file_filt3D = os.path.join(output_dir,name[:-10]+'_landmarksFiltered3D.csv')
                    if not os.path.exists(landmark_file_filt3D):
                        get3dlandmarks_video(depth_file, landmark_file_filt)

                except: 
                    continue
            


NF01_02_20170515_BBP_NORMAL_color.avi
error File: NF01_02_20170515_BBP_NORMAL_depth.avi
frames video = 1274 | Frames landmarks file = 1020

Video cannot be processed



In [18]:
pwd

'/home/deniz/Code/DataQuality'

In [ ]:
busted_als = [AF16_02_RST_REST.bag, AF19_02_DDK_PA.bag, AF18_02_RST_REST_1.bag]

In [ ]:
if 0:

    path_v = r'/home/deniz/Code/DataQuality/test_video'
    ext_v=('.bag', '.rssdk')
    Files_v = os.listdir(path_v)           
    Files_v = [i for i in Files_v if i.endswith(ext_v)]

    print(len(Files_v))

    for i in range(0, len(Files_v)):
        print(Files_v[i])